In [1]:
import tensorflow as tf

In [7]:
import tkinter as tk
from tkinter import filedialog
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Define the CTCLayer
class CTCLayer(tf.keras.layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = tf.keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions.
        return y_pred

# Load the saved model
model_path = 'text_recognizer.h5'  # replace with your model path
model = load_model(model_path, custom_objects={'CTCLayer': CTCLayer})
prediction_model = tf.keras.models.Model(
  model.get_layer(name="image").input, model.get_layer(name="dense2").output
)

# Define the image size and the function for image preprocessing
image_width = 128
image_height = 32

def distortion_free_resize(image, img_size):
  # similar to your previous distortion_free_resize function
  w, h = img_size
  image = tf.image.resize(image, size=(h, w), preserve_aspect_ratio=True)

  # Check tha amount of padding needed to be done.
  pad_height = h - tf.shape(image)[0]
  pad_width = w - tf.shape(image)[1]

  # only necessary if you want to do same amount of padding on both sides.
  if pad_height % 2 != 0:
    height = pad_height // 2
    pad_height_top = height +1
    pad_height_bottom = height
  else:
    pad_height_top = pad_height_bottom = pad_height // 2

  if pad_width % 2 != 0:
    width = pad_width // 2
    pad_width_left = width + 1
    pad_width_right = width
  else:
    pad_width_left = pad_width_right = pad_width // 2

  image = tf.pad(
      image, paddings=[
          [pad_height_top, pad_height_bottom],
          [pad_width_left, pad_width_right],
          [0, 0],
      ],
  )
  image = tf.transpose(image, perm=[1,0,2])
  image = tf.image.flip_left_right(image)
  return image  # replace with your function body

def preprocess_image(image_path, img_size=(image_width, image_height)):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_png(image, 1)
  image = distortion_free_resize(image, img_size)
  image = tf.cast(image, tf.float32) / 255.0
  return image

# Define the function for prediction
def predict_on_new_image(image_path):
  image = preprocess_image(image_path)
  image_batch = tf.expand_dims(image, axis=0)
  preds = prediction_model.predict(image_batch)
  pred_texts = decode_batch_predictions(preds)
  return pred_texts[0]

# Create a new tkinter window
window = tk.Tk()

# Function to open the file dialog and allow user to select an image
def upload_image():
  file_path = filedialog.askopenfilename()
  prediction = predict_on_new_image(file_path)
  print('Prediction:', prediction)

# Create a button that will call the upload_image function when clicked
upload_button = tk.Button(window, text='Upload image', command=upload_image)
upload_button.pack()

# Run the tkinter event loop
window.mainloop()


ValueError: Could not interpret initializer identifier: {'module': 'keras.initializers', 'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}, 'registered_name': None}